In [24]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import csv

In [25]:
train = pd.read_csv('data//roberta//roberta.train.csv')
train_x = train.iloc[:,:-1]
train_y = train.iloc[:,-1:]

train_tfidf = pd.read_csv('data//tfidf//tfidf.train.csv')
train_x_tfidf = train_tfidf.iloc[:,:-1]
train_y_tfidf = train_tfidf.iloc[:,-1:]

test = pd.read_csv('data//roberta//roberta.test.csv')
test_x = test.iloc[:,:-1]
test_y = test.iloc[:,-1:]

test_tfidf = pd.read_csv('data//tfidf//tfidf.test.csv')
test_x_tfidf = test_tfidf.iloc[:,:-1]
test_y_tfidf = test_tfidf.iloc[:,-1:]

eval_df = pd.read_csv('data//roberta//roberta.eval.anon.csv')
eval_df = eval_df.iloc[:,:-1]

eval_df_tfidf = pd.read_csv('data//tfidf//tfidf.eval.anon.csv')
eval_df_tfidf = eval_df_tfidf.iloc[:,:-1]

In [26]:
train_y = train_y.replace(0, -1)
train_y_tfidf = train_y_tfidf.replace(0, -1)
test_y = test_y.replace(0, -1)
test_y_tfidf = test_y_tfidf.replace(0, -1)

In [27]:
for i in range(train_x.shape[1]):
    col = train_x.columns[i]
    train_x.rename(columns = {col:'roberta_'+col}, inplace = True)
    
for i in range(train_x_tfidf.shape[1]):
    col = train_x_tfidf.columns[i]
    train_x_tfidf.rename(columns = {col:'tfidf_'+col}, inplace = True)
    
for i in range(test_x.shape[1]):
    col = test_x.columns[i]
    test_x.rename(columns = {col:'roberta_'+col}, inplace = True)
    
for i in range(test_x_tfidf.shape[1]):
    col = test_x_tfidf.columns[i]
    test_x_tfidf.rename(columns = {col:'tfidf_'+col}, inplace = True)

for i in range(eval_df.shape[1]):
    col = eval_df.columns[i]
    eval_df.rename(columns = {col:'roberta_'+col}, inplace = True)
    
for i in range(eval_df_tfidf.shape[1]):
    col = eval_df_tfidf.columns[i]
    eval_df_tfidf.rename(columns = {col:'tfidf_'+col}, inplace = True)

col = train_x_tfidf[train_x_tfidf.columns]
train_x = train_x.join(col)

col = test_x_tfidf[test_x_tfidf.columns]
test_x = test_x.join(col)

col = eval_df_tfidf[eval_df_tfidf.columns]
eval_df = eval_df.join(col)

In [28]:
# Create three different classifiers
clf1 = DecisionTreeClassifier(max_depth=4)
clf2 = SVC(kernel='linear',random_state=65)
clf3 = LogisticRegression(max_iter=1250,random_state=65)

In [29]:
# Create a voting classifier
voting_clf = VotingClassifier(estimators=[('dt', clf1), ('svc', clf2), ('lr', clf3)], voting='hard')

In [30]:
# Fit the voting classifier on the training data
voting_clf.fit(train_x, train_y)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


VotingClassifier(estimators=[('dt', DecisionTreeClassifier(max_depth=4)),
                             ('svc', SVC(kernel='linear', random_state=65)),
                             ('lr',
                              LogisticRegression(max_iter=1250,
                                                 random_state=65))])

In [31]:
# Make predictions on the testing data
pred = voting_clf.predict(train_x)

# Evaluate the accuracy of the voting classifier
accuracy = accuracy_score(train_y, pred)
print("Voting classifier training accuracy:", accuracy)

Voting classifier training accuracy: 0.9132336343115124


In [32]:
# Make predictions on the testing data
pred = voting_clf.predict(test_x)

# Evaluate the accuracy of the voting classifier
accuracy = accuracy_score(test_y, pred)
print("Voting classifier test accuracy:", accuracy)

Voting classifier test accuracy: 0.8215931533903884


In [33]:
eval_pred = voting_clf.predict(eval_df)

In [34]:
file = open('3_voting_classifier_roberta_tfidf_new_parameters.csv', 'w', newline ='')

with file:
    header = ['example_id', 'label']
    writer = csv.DictWriter(file, fieldnames = header)

    writer.writeheader()
    for i in range(len(eval_pred)):
        if eval_pred[i] == -1:
            val = 0
        else:
            val = 1
        writer.writerow({'example_id' : i, 'label': val})